In [6]:
import numpy as np
from tqdm import tqdm
import math
from matplotlib import pyplot as plt

In [7]:
class JordanNetwork:
    def __init__(self, weights=None, input_size=0, reccurent_size=0):
        self.input_size = input_size
        self.input = np.zeros((input_size, 1))
        self.reccurent = np.zeros((reccurent_size, 1))
        self.sum = np.zeros((reccurent_size, 1))
        self.output = np.zeros((1, 1))
        self.output_prev = np.zeros((1, 1))
        
        self.input_rec_w = np.random.rand(reccurent_size, input_size) / 5 - 0.1
        self.output_rec_w = np.random.rand(reccurent_size, 1) / 5 - 0.1
        self.rec_output_w = np.random.rand(1, reccurent_size) / 5 - 0.1
        self.T_rec = np.random.rand(reccurent_size, 1) / 5 - 0.1
        self.T_output = np.random.rand(1, 1) / 5 - 0.1
        
    
    def generate_sequence_with_result(self, sequence, size):
        sequences = []
        target = []
        for i in range(len(sequence) - size - 1):
            sequences.append(sequence[i:i + size])
            target.append(sequence[i+size])
        return np.array(sequences), np.array(target)
    
    def asinh_deriv(self, value):
        return 1/(value**2 + 1)**0.5
    
    def learn(self, sequence, learn_coef = 0.0001, cycle_num = 1):
        dataset, target = self.generate_sequence_with_result(sequence, self.input_size)
        for curr_cycle in tqdm(range(cycle_num)):
            for seq_num in range(len(dataset)):
                self.input = np.array([dataset[seq_num]]).T
                self.sum = self.input_rec_w @ self.input + self.output_rec_w @ self.output_prev - self.T_rec
                self.reccurent = np.arcsinh(self.sum)
                self.output = self.rec_output_w @ self.reccurent - self.T_output
                
                loss = self.output[0][0] - target[seq_num]
                self.input_rec_w  -= learn_coef * loss * (self.rec_output_w.T * self.asinh_deriv(self.sum)) @ self.input.T
                self.output_rec_w -= learn_coef * loss * (self.rec_output_w.T * self.asinh_deriv(self.sum)) @ self.output_prev.T
                self.T_rec        += learn_coef * loss * self.rec_output_w.T * self.asinh_deriv(self.sum)
                self.T_output     += learn_coef * loss
                self.rec_output_w -= learn_coef * loss * self.reccurent.T
                
                self.output_prev = self.output
            self.output_prev = np.zeros_like(self.output)
            
    def predict(self, input, predict_size):
        rnn = len(self.input_rec_w)
        output = len(self.rec_output_w)
        
        predicted_continuation = []
        
        for i in tqdm(range(len(input) - self.input_size + predict_size)):
            # a step of a neural network
            self.input = np.array([input[i:i + self.input_size]]).T
            self.sum = self.input_rec_w @ self.input + self.output_rec_w @ self.output_prev - self.T_rec
            self.reccurent = np.arcsinh(self.sum)
            self.output = self.rec_output_w @ self.reccurent - self.T_output
            
            self.output_prev = self.output
            
            if i >= len(input) - self.input_size: 
                input.append(self.output[0][0])
                predicted_continuation.append(self.output[0][0])
        
        return predicted_continuation

In [8]:
def generate_fib(num):
    result = [1,1]
    fib1 = 1
    fib2 = 1
    i = 0
    while i < num - 2:
        fib_sum = fib1 + fib2
        fib1 = fib2
        fib2 = fib_sum
        i = i + 1
        result.append(fib_sum)
    return result

In [17]:
NN = JordanNetwork(None, 3, 9)
sequence = generate_fib(11)
NN.learn(sequence, learn_coef=0.001, cycle_num=50000)
print(NN.predict([1,1,2,3],6))

100%|██████████| 7/7 [00:00<00:00, 35288.62it/s]

[5.0034862807131475, 7.989668457293433, 13.006972991684925, 21.00621465638054, 34.022094632516314, 55.012363758486714]


In [18]:
NN = JordanNetwork(None, 3, 7)
sequence = [x**2 for x in range(10)]
NN.learn(sequence, learn_coef=0.00001, cycle_num=130000)
print(NN.predict([0,1,4,9], 5))

100%|██████████| 6/6 [00:00<00:00, 21219.08it/s]

[15.991467415494654, 25.01155458346667, 35.97349944138743, 49.04567520098215, 63.95454611733168]


In [23]:
NN = JordanNetwork(None, 5, 11)
sequence = [-1, 0, 1, -1, 0, 1, -1, 0, 1, -1, 0]
NN.learn(sequence, learn_coef=0.00001, cycle_num=130000)
print(NN.predict([-1, 0, 1, -1, 0, 1], 5))

100%|██████████| 6/6 [00:00<00:00, 30430.26it/s]

[-0.9999295109095138, 0.0008193140378921289, 0.9996261322240949, -1.0000006196464315, 0.0013616338957999174]
